In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextIteratorStreamer
import threading

/home/azar/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --- Step 1: Define the local directory ---
# Make sure this path points to where you saved the model
local_model_path = "./gemma-3-270m-it-local"

# --- Step 2: Load the tokenizer and model from the local directory ---
print(f"Loading tokenizer from: {local_model_path}")
tokenizer = AutoTokenizer.from_pretrained(local_model_path)

print(f"Loading model from: {local_model_path}")
model = AutoModelForCausalLM.from_pretrained(local_model_path)

Loading tokenizer from: ./gemma-3-270m-it-local
Loading model from: ./gemma-3-270m-it-local


In [11]:
def chat_with_model_stream(messages, tokenizer, model, max_new_tokens=4000):
    # Prepare inputs
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to(model.device)

    # Set up streamer for real-time decoding
    streamer = TextIteratorStreamer(tokenizer, skip_special_tokens=True)

    # Run generation in a background thread so we can stream tokens
    generation_kwargs = dict(**inputs, streamer=streamer, max_new_tokens=max_new_tokens)
    thread = threading.Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    # Stream tokens as they are generated
    response = ""
    for new_text in streamer:
        print(new_text, end="", flush=True)  # live printing
        response += new_text

    return response

In [6]:
# Example usage
messages = [{"role": "user", "content": "Who are you?"}]
result = chat_with_model_stream(messages, tokenizer, model)


Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


user
Who are you?
model


I am Gemma, an open-weights AI assistant. I am a large language model created by the Gemma team.


In [12]:
# Example usage
messages = [{"role": "user", "content": "what do you know about algeria ?"}]
chat_with_model_stream(messages, tokenizer, model)


Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


user
what do you know about algeria ?
model
Algeria is a fascinating country with a rich history, culture, and modern landscape. Here's a summary of what I know about it:

*   **Geography:** Algeria is a large country, with significant geographical features, including the Sahara Desert, the Moroccan Highlands, and the Rif Mountains.
*   **History:** Algeria has a long and complex history, with its origins as a kingdom under various empires. Its history is marked by conquests, wars, and political developments.
*   **Culture:** Algeria is known for its diverse culture, including Arabic language, music, art, and cuisine.
*   **Economy:** Algeria is a major exporter of oil and gas, with a growing focus on renewable energy and sustainable development.
*   **Politics:** Algeria is a parliamentary republic with a significant degree of political autonomy.

To give you a more specific answer, I need more information about your interests. For example, are you interested in:

*   **History?**
*  

"user\nwhat do you know about algeria ?\nmodel\nAlgeria is a fascinating country with a rich history, culture, and modern landscape. Here's a summary of what I know about it:\n\n*   **Geography:** Algeria is a large country, with significant geographical features, including the Sahara Desert, the Moroccan Highlands, and the Rif Mountains.\n*   **History:** Algeria has a long and complex history, with its origins as a kingdom under various empires. Its history is marked by conquests, wars, and political developments.\n*   **Culture:** Algeria is known for its diverse culture, including Arabic language, music, art, and cuisine.\n*   **Economy:** Algeria is a major exporter of oil and gas, with a growing focus on renewable energy and sustainable development.\n*   **Politics:** Algeria is a parliamentary republic with a significant degree of political autonomy.\n\nTo give you a more specific answer, I need more information about your interests. For example, are you interested in:\n\n*   **

In [37]:
import requests

BASE_URL = "http://127.0.0.1:5000"

def stream_generate(prompt):
    response = requests.post(f"{BASE_URL}/generate-stream", json={"prompt": prompt}, stream=True)
    if response.status_code == 200:
        for chunk in response.iter_content(chunk_size=1, decode_unicode=True):
            if chunk: print(chunk, end='', flush=True)

def stop_generation():
    return requests.post(f"{BASE_URL}/stop-generation").json()

def check_status():
    return requests.get(f"{BASE_URL}/generation-status").json()

In [39]:
stream_generate("What is Algeria , be more specific?")

user
What is Algeria , be more specific?
model
Algeria is a large and diverse country with a rich history, culture, and a vibrant economy. It's known for its rich natural resources, including oil, gas, minerals, and diverse agricultural products. Algeria is also a significant player in the global energy market and a major economic power in the region.
